In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

from ray.rllib.agents.trainer import Trainer, with_common_config
from ray.rllib.utils.annotations import override


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# yapf: disable
# __sphinx_doc_begin__
class RandomAgent(Trainer):
    """Policy that takes random actions and never learns."""

    _name = "RandomAgent"
    _default_config = with_common_config({
        "rollouts_per_iteration": 10,
    })

    @override(Trainer)
    def _init(self, config, env_creator):
        self.env = env_creator(config["env_config"])

    @override(Trainer)
    def _train(self):
        rewards = []
        steps = 0
        for _ in range(self.config["rollouts_per_iteration"]):
            obs = self.env.reset()
            done = False
            reward = 0.0
            while not done:
                action = self.env.action_space.sample()
                obs, r, done, info = self.env.step(action)

                reward += r
                steps += 1
            rewards.append(reward)
        return {
            "episode_reward_mean": np.mean(rewards),
            "timesteps_this_iter": steps,
        }

class VIAgent(Trainer):
    """Value Iteration.
    #TODO Make it Generalized PI.
    """

    _name = "VIAgent"
    _default_config = with_common_config({
        "tolerance": 0.01,
        "discount_factor": 0.5,
        "rollouts_per_iteration": 10,
        "episode_length": 200,
        # "lr": 0.5
    })

    @override(Trainer)
    def _init(self, config, env_creator):
        self.env = env_creator(config["env_config"])
        self.V = np.zeros(self.env.observation_space.n)
        self.policy = np.zeros(self.env.observation_space.n, dtype=int)
        self.policy[:] = -1 #IMP # To avoid initing it to a value within action_space range

    @override(Trainer)
    def _train(self):
        max_diff = np.inf # Maybe keep a state variable so that we don't need to update every train iteration??
        state_space_size = self.env.observation_space.n
        gamma = self.config["discount_factor"]
        total_iterations = 0
        while max_diff > self.config["tolerance"]:
            total_iterations += 1
            for s in range(state_space_size):
                # print("self.V[:]", s, max_diff, self.V, [self.env.R(s, a) for a in range(self.env.action_space.n)], self.policy[s])
                self.V_old = self.V.copy() # Is this asynchronous? V_old should be held constant for all states in the for loop?
                # print([self.env.R(s, a) for a in range(self.env.action_space.n)], [gamma * self.V[self.env.P(s, a)] for a in range(self.env.action_space.n)], [self.env.R(s, a) + gamma * self.V[self.env.P(s, a)] for a in range(self.env.action_space.n)])
                self.policy[s] = np.argmax([self.env.R(s, a) + gamma * self.V[self.env.P(s, a)] for a in range(self.env.action_space.n)])
                self.V[s] = np.max([self.env.R(s, a) + gamma * self.V[self.env.P(s, a)] for a in range(self.env.action_space.n)]) # We want R to be a callable function, so I guess we have to keep a for loop here??
                # print("self.V, self.V_old, self.policy[s]", self.V, self.V_old, self.policy[s], self.env.P(s, self.policy[s]))

                max_diff = np.max(np.absolute(self.V_old - self.V))
        # import time
        # time.sleep(2)
#         for s in range(state_space_size):
#             print("FINAL self.V[:]", s, max_diff, self.V[:], [self.env.R(s, a) for a in range(self.env.action_space.n)])

        print("Total iterations:", total_iterations)
        rewards = []
        steps = 0
        for _ in range(self.config["rollouts_per_iteration"]):
            obs = self.env.reset()
            done = False
            reward = 0.0
            for _ in range(self.config["episode_length"]):
                action = self.policy[obs]
                obs, r, done, info = self.env.step(action)

                reward += r
                steps += 1
            rewards.append(reward)
        return {
            "episode_reward_mean": np.mean(rewards),
            "timesteps_this_iter": steps,
        }


In [3]:
import ray
from ray import tune
from ray.rllib.utils.seed import seed as rllib_seed
import rl_toy
from rl_toy.envs import RLToyEnv
from ray.tune.registry import register_env
register_env("RLToy-v0", lambda config: RLToyEnv(config))

# rllib_seed(0, 0, 0)
ray.init()


2019-08-17 21:51:40,487	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-17_21-51-40_487449_24872/logs.
2019-08-17 21:51:40,608	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:28790 to respond...
2019-08-17 21:51:40,726	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:56305 to respond...
2019-08-17 21:51:40,729	INFO services.py:809 -- Starting Redis shard with 6.72 GB max memory.
2019-08-17 21:51:40,754	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-17_21-51-40_487449_24872/logs.
2019-08-17 21:51:40,756	WARNING services.py:1330 -- WARNING: The default object store size of 10.08 GB will use more than 50% of the available memory on this node (17.17 GB). Consider setting the object store memory manually to a smaller size to avoid memory contention with other applications.
2019-08-17 21:51:40,757	INFO services.py:1475 -- Starting the Plasma object store with 10.08 GB memor

{'node_ip_address': '10.5.150.104',
 'redis_address': '10.5.150.104:28790',
 'object_store_address': '/tmp/ray/session_2019-08-17_21-51-40_487449_24872/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-08-17_21-51-40_487449_24872/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-08-17_21-51-40_487449_24872'}

In [19]:
# stats = {}
# aaaa = 3

# fout = open('rl_stats_temp.csv', 'a') #hardcoded
# fout.write('# basename, n_points, n_features, n_trees ')


def on_train_result(info):
    print("#############trainer.train() result: {} -> {} episodes".format(
        info["trainer"], info["result"]["episodes_this_iter"]), info)
    # you can mutate the result dict to add new fields to return
    stats['episode_len_mean'] = info['result']['episode_len_mean']
#     print("++++++++", aaaa, stats)
    state_space_size = info["result"]["config"]["env_config"]["state_space_size"]
#     action_space_size = 
    fout = open('rl_stats_temp.csv', 'a') #hardcoded
    fout.write('# basename, n_points, n_features, n_trees ' + str(state_space_size) + ' ' + str() + ' ' + str() + ' ' + str() + '\n')
    fout.close()

    info["result"]["callback_ok"] = True
    

# tune.run(
#     RandomAgent,
#     stop={
#         "timesteps_total": 20000,
#           },
#     config={
#       "rollouts_per_iteration": 10,
#       "env": "RLToy-v0",
#       "env_config": {
#         'state_space_type': 'discrete',
#         'action_space_type': 'discrete',
#         'state_space_size': 16,
#         'action_space_size': 16,
#         'generate_random_mdp': True,
#         'delay': 6,
#         'sequence_length': 1,
#         'reward_density': 0.25,
#         'terminal_state_density': 0.25
#         },
#     },
# )

# tune.run(
#     VIAgent,
#     stop={
#         "timesteps_total": 20000,
#           },
#     config={
#         "tolerance": 0.01,
#         "discount_factor": 0.99,
#         "rollouts_per_iteration": 10,
#       "env": "RLToy-v0",
#       "env_config": {
#         'state_space_type': 'discrete',
#         'action_space_type': 'discrete',
#         'state_space_size': 10,
#         'action_space_size': 10,
#         'generate_random_mdp': True,
#         'delay': 0,
#         'sequence_length': 1,
#         'reward_density': 0.25,
#         'terminal_state_density': 0.25
#         },
#     },
# )


tune.run(
    "DQN",
    stop={
        "timesteps_total": 20000,
          },
    config={
      "adam_epsilon": 0.00015,
      "beta_annealing_fraction": 1.0,
      "buffer_size": 1000000,
      "double_q": False,
      "dueling": False,
      "env": "RLToy-v0",
      "env_config": {
        'state_space_type': 'discrete',
        'action_space_type': 'discrete',
        'state_space_size': 16,
        'action_space_size': 16,
        'generate_random_mdp': True,
        'delay': 6,
        'sequence_length': 1,
        'reward_density': 0.25,
        'terminal_state_density': 0.25
        },
      "exploration_final_eps": 0.01,
      "exploration_fraction": 0.1,
      "final_prioritized_replay_beta": 1.0,
      "hiddens": [
        256
      ],
      "learning_starts": 2000,
      "lr": 6.25e-05, # "lr": grid_search([1e-2, 1e-4, 1e-6]),
      "n_step": 1,
      "noisy": False,
      "num_atoms": 1,
      "prioritized_replay": False,
      "prioritized_replay_alpha": 0.5,
      "sample_batch_size": 4,
      "schedule_max_timesteps": 20000,
      "target_network_update_freq": 80,
      "timesteps_per_iteration": 100,
      "train_batch_size": 32,
        
              "callbacks": {
#                 "on_episode_start": tune.function(on_episode_start),
#                 "on_episode_step": tune.function(on_episode_step),
#                 "on_episode_end": tune.function(on_episode_end),
#                 "on_sample_end": tune.function(on_sample_end),
                "on_train_result": tune.function(on_train_result),
#                 "on_postprocess_traj": tune.function(on_postprocess_traj),
            },
    },
)


2019-08-18 00:41:15,614	INFO trial_runner.py:176 -- Starting a new experiment.
2019-08-18 00:41:15,621	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 17.0/33.6 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/4 CPUs, 0/0 GPUs
Memory usage on this node: 17.0/33.6 GB
Result logdir: /home/rajanr/ray_results/DQN
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - DQN_RLToy-v0_0:	RUNNING

(pid=515) WARNING:tensorflow:From /home/rajanr/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/compat/compat.py:175: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=515) Instructions for updating:
(pid=515) non-resource variables are not supported in the long term
(pid=515) Inited terminal states to: [15 14 13 12] total 4
(pid=515) specific_sequence that will be rewarded [1]
(pid=515) specific_sequence that will be rewarded [5]
(pid=515) specific_sequence that will be rewarded [0]
(pid=515) Total no. of sequences re

Result for DQN_RLToy-v0_0:
  callback_ok: true
  custom_metrics: {}
  date: 2019-08-18_00-41-22
  done: false
  episode_len_mean: 3.475836431226766
  episode_reward_max: 5.0
  episode_reward_mean: 0.895910780669145
  episode_reward_min: 0.0
  episodes_this_iter: 269
  episodes_total: 269
  experiment_id: 52abb92d04254715928ae3620e2b94a8
  hostname: mlstaff04
  info:
    grad_time_ms: .nan
    learner: {}
    max_exploration: 1.0
    min_exploration: 1.0
    num_steps_sampled: 936
    num_steps_trained: 0
    num_target_updates: 11
    opt_peak_throughput: 0.0
    opt_samples: .nan
    replay_time_ms: .nan
    sample_time_ms: 3.46
    update_time_ms: 0.001
  iterations_since_restore: 1
  node_ip: 10.5.150.104
  num_healthy_workers: 0
  off_policy_estimator: {}
  pid: 515
  policy_reward_mean: {}
  sampler_perf:
    mean_env_wait_ms: 0.059969524436470294
    mean_inference_ms: 0.5707908719937089
    mean_processing_ms: 0.21816139669021387
  time_since_restore: 1.0066626071929932
  time_t

(pid=515) 2019-08-18 00:41:23,250	INFO rollout_worker.py:575 -- Training on concatenated sample batches:
(pid=515) 
(pid=515) { 'count': 32,
(pid=515)   'policy_batches': { 'default_policy': { 'data': { 'actions': np.ndarray((32,), dtype=int64, min=0.0, max=15.0, mean=7.625),
(pid=515)                                                     'batch_indexes': np.ndarray((32,), dtype=int64, min=-1.0, max=-1.0, mean=-1.0),
(pid=515)                                                     'dones': np.ndarray((32,), dtype=bool, min=0.0, max=1.0, mean=0.344),
(pid=515)                                                     'new_obs': np.ndarray((32, 16), dtype=float32, min=0.0, max=1.0, mean=0.062),
(pid=515)                                                     'obs': np.ndarray((32, 16), dtype=float32, min=0.0, max=1.0, mean=0.062),
(pid=515)                                                     'rewards': np.ndarray((32,), dtype=int64, min=0.0, max=1.0, mean=0.312),
(pid=515)                             

(pid=515) #############trainer.train() result: <ray.rllib.agents.trainer_template.DQN object at 0x7f23b2207978> -> 50 episodes {'trainer': <ray.rllib.agents.trainer_template.DQN object at 0x7f23b2207978>, 'result': {'episode_reward_max': 3.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 0.71, 'episode_len_mean': 7.8, 'episodes_this_iter': 50, 'policy_reward_mean': {}, 'custom_metrics': {}, 'sampler_perf': {'mean_env_wait_ms': 0.06167557679553973, 'mean_processing_ms': 0.21702204265660222, 'mean_inference_ms': 0.5702078057393867}, 'off_policy_estimator': {}, 'info': {'min_exploration': 0.010000000000000009, 'max_exploration': 0.010000000000000009, 'num_target_updates': 33, 'num_steps_trained': 6400, 'num_steps_sampled': 2800, 'sample_time_ms': 4.616, 'replay_time_ms': 2.502, 'grad_time_ms': 2.419, 'update_time_ms': 0.003, 'opt_peak_throughput': 13227.331, 'opt_samples': 32.0, 'learner': {'default_policy': {'cur_lr': 6.25000029685907e-05, 'mean_q': 0.5450195, 'min_q': -0.122599974, 

...

Result for DQN_RLToy-v0_0:
  callback_ok: true
  custom_metrics: {}
  date: 2019-08-18_00-41-57
  done: false
  episode_len_mean: 139.58
  episode_reward_max: 1211.0
  episode_reward_mean: 62.53
  episode_reward_min: 0.0
  episodes_this_iter: 0
  episodes_total: 744
  experiment_id: 52abb92d04254715928ae3620e2b94a8
  hostname: mlstaff04
  info:
    grad_time_ms: 2.237
    learner:
      default_policy:
        cur_lr: 6.25000029685907e-05
        max_q: 61.372074127197266
        mean_q: 58.63371658325195
        mean_td_error: -0.23699043691158295
        min_q: 6.222259998321533
        model: {}
    max_exploration: 0.010000000000000009
    min_exploration: 0.010000000000000009
    num_steps_sampled: 17176
    num_steps_trained: 121408
    num_target_updates: 204
    opt_peak_throughput: 14303.437
    opt_samples: 32.0
    replay_time_ms: 3.376
    sample_time_ms: 3.666
    update_time_ms: 0.002
  iterations_since_restore: 36
  node_ip: 10.5.150.104
  num_healthy_workers: 0
  off_po

(pid=515) #############trainer.train() result: <ray.rllib.agents.trainer_template.DQN object at 0x7f23b2207978> -> 1 episodes {'trainer': <ray.rllib.agents.trainer_template.DQN object at 0x7f23b2207978>, 'result': {'episode_reward_max': 1211.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 70.12, 'episode_len_mean': 157.12, 'episodes_this_iter': 1, 'policy_reward_mean': {}, 'custom_metrics': {}, 'sampler_perf': {'mean_env_wait_ms': 0.062033557148359, 'mean_processing_ms': 0.20237771370318175, 'mean_inference_ms': 0.5670882314107144}, 'off_policy_estimator': {}, 'info': {'min_exploration': 0.010000000000000009, 'max_exploration': 0.010000000000000009, 'num_target_updates': 215, 'num_steps_trained': 128800, 'num_steps_sampled': 18100, 'sample_time_ms': 3.619, 'replay_time_ms': 2.939, 'grad_time_ms': 2.145, 'update_time_ms': 0.002, 'opt_peak_throughput': 14918.385, 'opt_samples': 32.0, 'learner': {'default_policy': {'cur_lr': 6.25000029685907e-05, 'mean_q': 64.10534, 'min_q': 61.91525

(pid=515) #############trainer.train() result: <ray.rllib.agents.trainer_template.DQN object at 0x7f23b2207978> -> 0 episodes {'trainer': <ray.rllib.agents.trainer_template.DQN object at 0x7f23b2207978>, 'result': {'episode_reward_max': 1211.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 72.7, 'episode_len_mean': 163.15, 'episodes_this_iter': 0, 'policy_reward_mean': {}, 'custom_metrics': {}, 'sampler_perf': {'mean_env_wait_ms': 0.062044891471392895, 'mean_processing_ms': 0.20168397268716354, 'mean_inference_ms': 0.5668711526235305}, 'off_policy_estimator': {}, 'info': {'min_exploration': 0.010000000000000009, 'max_exploration': 0.010000000000000009, 'num_target_updates': 226, 'num_steps_trained': 136416, 'num_steps_sampled': 19052, 'sample_time_ms': 3.481, 'replay_time_ms': 2.797, 'grad_time_ms': 2.147, 'update_time_ms': 0.002, 'opt_peak_throughput': 14903.642, 'opt_samples': 32.0, 'learner': {'default_policy': {'cur_lr': 6.25000029685907e-05, 'mean_q': 58.53141, 'min_q': 0.9802

Result for DQN_RLToy-v0_0:
  callback_ok: true
  custom_metrics: {}
  date: 2019-08-18_00-42-03
  done: true
  episode_len_mean: 175.43
  episode_reward_max: 1211.0
  episode_reward_mean: 78.49
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 748
  experiment_id: 52abb92d04254715928ae3620e2b94a8
  hostname: mlstaff04
  info:
    grad_time_ms: 2.046
    learner:
      default_policy:
        cur_lr: 6.25000029685907e-05
        max_q: 69.28202819824219
        mean_q: 61.954063415527344
        mean_td_error: 0.09763866662979126
        min_q: -0.3660565912723541
        model: {}
    max_exploration: 0.010000000000000009
    min_exploration: 0.010000000000000009
    num_steps_sampled: 20008
    num_steps_trained: 144064
    num_target_updates: 238
    opt_peak_throughput: 15639.993
    opt_samples: 32.0
    replay_time_ms: 2.711
    sample_time_ms: 3.511
    update_time_ms: 0.003
  iterations_since_restore: 42
  node_ip: 10.5.150.104
  num_healthy_workers: 0
  off_po

(pid=515) #############trainer.train() result: <ray.rllib.agents.trainer_template.DQN object at 0x7f23b2207978> -> 1 episodes {'trainer': <ray.rllib.agents.trainer_template.DQN object at 0x7f23b2207978>, 'result': {'episode_reward_max': 1211.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 78.49, 'episode_len_mean': 175.43, 'episodes_this_iter': 1, 'policy_reward_mean': {}, 'custom_metrics': {}, 'sampler_perf': {'mean_env_wait_ms': 0.06205245643800219, 'mean_processing_ms': 0.20097573154931517, 'mean_inference_ms': 0.5666116418198061}, 'off_policy_estimator': {}, 'info': {'min_exploration': 0.010000000000000009, 'max_exploration': 0.010000000000000009, 'num_target_updates': 238, 'num_steps_trained': 144064, 'num_steps_sampled': 20008, 'sample_time_ms': 3.511, 'replay_time_ms': 2.711, 'grad_time_ms': 2.046, 'update_time_ms': 0.003, 'opt_peak_throughput': 15639.993, 'opt_samples': 32.0, 'learner': {'default_policy': {'cur_lr': 6.25000029685907e-05, 'mean_q': 61.954063, 'min_q': -0.36

In [5]:
state_space_sizes = [2**i for i in range(1,6)]
action_space_sizes = [2**i for i in range(1,6)]
delays = [0] + [2**i for i in range(5)]
sequence_lengths = [i for i in range(1,6)]
reward_densities = np.linspace(0.0, 1.0, num=5)
# make_reward_dense = [True, False]
terminal_state_densities = np.linspace(0.1, 1.0, num=5)

print(state_space_sizes, action_space_sizes, delays, sequence_lengths, reward_densities, terminal_state_densities)


[2, 4, 8, 16, 32] [2, 4, 8, 16, 32] [0, 1, 2, 4, 8, 16] [1, 2, 3, 4, 5] [0.   0.25 0.5  0.75 1.  ] [0.1   0.325 0.55  0.775 1.   ]


In [17]:
stats
fout = open('rl_stats_temp.csv', 'a') #hardcoded
fout.write('# basename, n_points, n_features, n_trees ')

fout.close()
